In [1]:
%load_ext autoreload
%autoreload 2

import dotenv
import json
import langchain
from pydantic.dataclasses import dataclass, Field

import interlab
from interlab import actor, context
from interlab.context import Context, with_context

import openai
import os
dotenv.load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

import time

from interlab.lang_models import WebConsoleModel
from interlab.actor import WebConsoleActor

In [2]:
name_a = input('Participant A, please enter your name:')
name_b = input('Participant B, please enter your name:')

Participant A, please enter your name: Lynn
Participant B, please enter your name: Nuora


In [3]:
#context browser
storage = context.FileStorage("logs") # Directory for storing contexts (structured logs)
storage.live_display(height=500)
# Alternatively, you can use storage.start_server() if you want to only open the storage in another browser tab

In [4]:
model = WebConsoleModel("Web console")
model.display(height=400)

In [5]:
import csv

#bot actor setup
e4_bot = langchain.chat_models.ChatOpenAI(model_name="gpt-4", temperature = 0.7)

bot_instructions = {}
with open("bot_instructions.csv", encoding='utf-8-sig') as csvfile:
    dict_reader = csv.DictReader(csvfile, escapechar='\\')
    
    # Initialize bot_instructions with keys from the first row
    bot_instructions = {key: [] for key in dict_reader.fieldnames}

    # Iterate through the rest of the file
    for row in dict_reader:
        for key, value in row.items():
            if value:
                bot_instructions[key].append(value)

initial_instructions = f'You are a discussion facilitator using the applied rationality method Double Crux to help participants find \'cruxes\' upon which the disagreement hinges. \
A crux for an individual is a fact that would change their conclusion in the overall disagreement. \
A double-crux is two symmetric, opposite cruxes: One crux is a fact that, if true, disproves one participant\'s belief. The second crux is the same fact that, if false, disproves the other participant\'s belief. \
Here is the premise: {name_a} and {name_b} must hold opposite beliefs. \
Your job is to discover a crux such that if {name_a} believed the crux was true (or false), then they would change their mind about their belief. \
Conversely, the opposite conclusion about the crux would change {name_b}\'s mind. \
These cruxes would constitute a double-crux. \
The cruxes should be more concrete, falsifiable, well-defined, and/or discoverable than the beliefs. \
Double-Crux differs from typical debates which are usually adversarial, and instead attempt to be a collaborative attempt to uncover the true underlying structure of the disagreement. \n\
Address and ask questions to only one participant at a time.'               

In [6]:
import enum
#pip3 install aenum
from aenum import extend_enum

class TurnID(enum.Enum):
    participant_a = name_a
    participant_b = name_b

#WARNING: bad practice way to set attributes - restart kernel if you update the csv file
class ConversationStep(enum.Enum):
    pass

for key in bot_instructions.keys():
    extend_enum(ConversationStep, key, key)

@dataclass
class ChatbotAction:
    analysis: str = Field(
        description="My hidden thoughts and reasoning, visible only to me.")
    next_turn: TurnID = Field(
        description="The single person who should reply next in the conversation. Switch between participants as often as possible.")
    reply: str = Field(
        description="My response, directed to the single participant who I chose to speak next.")

@dataclass
class DiscussionFlow:
    conversation_step: ConversationStep = Field(
        description="The next phase that this conversation should enter. \
        There are 3 choices:\n \
        Find crux = Try to find one of the participant's cruxes. You should do this at least once in every conversation, or multiple times until a crux is found that leads to a double crux.\n \
        Verify double crux = Check if the other participant shares a symmetric, opposite crux. You should do this after finding one of the participant's cruxes to see if it is a double crux.\n \
        Close discussion = End this conversation if you found a double crux, no double crux was found after a few tries, or if the conversation is out of control.\n")

In [7]:
#function to avoid sending api requests too quickly
def act(speaker, message=None, expected_type=None):
    for i in range(3):
        try:
            action_event = speaker.act(message, expected_type=expected_type)
            return action_event.data
        except:
            time.sleep(30)
        print('RATE LIMIT TIMEOUT')


def conversation(bot: actor.ActorWithMemory, max_turns = 50):
    #default
    result = "TIMEOUT"
    
    chat_history = []
    
    step, substep = 'Start', 0
    
    for i in range(1, max_turns + 1):
        if i%2 == 1:
            speaker = bot
            message = bot_instructions.get(step)[substep].format(participant_a = name_a, participant_b = name_b)
            expected_action = ChatbotAction
            
            action = act(
                speaker,
                message = message,
                expected_type=expected_action)
            assert isinstance(action, expected_action)
            
            hidden_thoughts = str(action.analysis)
            bot.observe(f'Conversation analysis: {hidden_thoughts}')
            
            reply = action.reply
            chat_history.append(
                f'{speaker.name}: {reply}')
            
        else:
            speaker = str(action.next_turn.value)
            reply = model.query(f'Chatbot: {reply}')
            chat_history.append(
                f'{speaker}: {reply}')
            bot.observe('\n'.join(chat_history[-2:]))
            substep += 1
            
        if substep == len(bot_instructions.get(step)):
            if step == 'Close discussion':
                break
            action = act(
                bot,
                message = None,
                expected_type=DiscussionFlow)
            assert isinstance(action, DiscussionFlow)
            step, substep = str(action.conversation_step.value), 0
            
        
    return result, chat_history

In [8]:
bot = actor.OneShotLLMActor("Double Crux Facilitator", e4_bot, f'{initial_instructions}')

with Context(f"double-crux", storage=storage) as c:
    r = conversation(bot)
    c.set_result(r)
    print(f"Done: {r}")

Done: ('TIMEOUT', ["Double Crux Facilitator: Hello Lynn, I'm here to facilitate our conversation today. My role is to help us uncover the true underlying structure of our disagreement using the Double Crux method. This isn't a debate, but a collaborative process to understand each other's perspectives better. I'm curious, are you open to the possibility of changing your mind today, assuming we find a crux that challenges your current belief?", 'Lynn: Yes, I am willing to change my mind and hoping at least one of us does because only one choice can be taken ultimately.', 'Double Crux Facilitator: Hello Nuora, as part of our discussion today, I want us to keep an open mind and be ready to consider different perspectives. Are you open to the possibility of changing your stance today, should a crux that challenges your current belief come to light?', 'Nuora: Yes, I am also open to changing my mind and in fact eager to do so. I want us to be a unified front when it comes time to make a fina